## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-02-02-14-50 +0000,nypost,"Man killed, four others injured in drive-by sh...",https://nypost.com/2025/09/01/us-news/man-kill...
1,2025-09-02-02-07-58 +0000,nypost,NYC woman busted for threatening to kill Presi...,https://nypost.com/2025/09/01/us-news/nyc-woma...
2,2025-09-02-01-41-07 +0000,nypost,Sexual assault counselor charged with heinous ...,https://nypost.com/2025/09/01/world-news/sexua...
3,2025-09-02-01-37-21 +0000,latimes,Photos: Labor Day celebrated,https://www.latimes.com/california/story/2025-...
4,2025-09-02-01-35-01 +0000,latimes,Police seek public's help with identifying Bur...,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,36
410,china,13
288,earthquake,12
287,afghanistan,11
232,dead,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
72,2025-09-01-20-19-52 +0000,nypost,India’s leader Modi cozies up to Putin and Xi ...,https://nypost.com/2025/09/01/world-news/india...,88
18,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...,77
64,2025-09-01-20-43-57 +0000,nypost,Trump awarding Presidential Medal of Freedom t...,https://nypost.com/2025/09/01/us-news/trump-aw...,72
22,2025-09-02-00-00-00 +0000,wsj,President Trump faces a challenging September ...,https://www.wsj.com/politics/policy/trumps-sep...,69
104,2025-09-01-17-25-00 +0000,wsj,The Trump administration has suspended visa ap...,https://www.wsj.com/politics/policy/trump-admi...,68


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
72,88,2025-09-01-20-19-52 +0000,nypost,India’s leader Modi cozies up to Putin and Xi ...,https://nypost.com/2025/09/01/world-news/india...
18,41,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...
158,41,2025-09-01-13-53-10 +0000,bbc,BBC reports from scene as Afghanistan earthqua...,https://www.bbc.com/news/videos/c3v3gg6zr3yo?a...
71,39,2025-09-01-20-20-50 +0000,nypost,"TikTok influencer and her family found dead, w...",https://nypost.com/2025/09/01/world-news/influ...
64,36,2025-09-01-20-43-57 +0000,nypost,Trump awarding Presidential Medal of Freedom t...,https://nypost.com/2025/09/01/us-news/trump-aw...
96,33,2025-09-01-18-23-50 +0000,nypost,Copper hits one-month peak on strong China fac...,https://nypost.com/2025/09/01/business/copper-...
98,32,2025-09-01-18-15-40 +0000,bbc,EU chief von der Leyen's plane hit by suspecte...,https://www.bbc.com/news/articles/c9d07z1439zo...
43,31,2025-09-01-21-53-22 +0000,nypost,Nevada sheriff issues desperate plea asking Bu...,https://nypost.com/2025/09/01/us-news/nevada-s...
143,30,2025-09-01-14-37-46 +0000,cbc,Israeli tanks advance in Gaza City as scholars...,https://www.cbc.ca/news/world/gaza-city-invasi...
17,29,2025-09-02-00-13-00 +0000,nypost,"Rep. Jerrold Nadler, 78, will not seek re-elec...",https://nypost.com/2025/09/01/us-news/rep-jerr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
